# Dataton BC 2018

*Dirección de Capacidades Analíticas y Gobierno de Información, Grupo Bancolombia*<br>
Esta versión: *2018-10-17*<br>
Documentación tablas

## Naturaleza de los datos

Los datos entregados en este reto corresponden a transacciones realizadas por clientes persona del banco vía [PSE](https://www.pse.com.co/inicio). Estas transacciones, a diferencia de las transacciones realizadas vía POS, no cuentan con un código [MCC](https://en.wikipedia.org/wiki/Merchant_category_code) atado a la transacción, que permite conocer la categoría de comercio a la que pertence el establecimiento de comercio donde se realiza la transacción. Adicionalmente, muchas de estas transferencias por PSE corresponden a transferencias de pagos de servicios públicos, seguros, colegios, arrendamientos, y otros gastos que pueden ser denominados como gastos grandes. En el marco de un sistema de gestión de finanzas personales, poder categorizar adecuadamente estas transacciones que se realizan por PSE es de suma importancia para contar con una foto completa de la actividad de gastos de los clientes. Para este reto, los equipos participantes tendrán acceso a una muestra de transacciones PSE que corresponden a algo más de 300 mil clientes (persona), seleccionados de manera aleatoria. La tabla de transacciones cuenta con 11.8 millones de registros (uno para cada transacción), realizados entre septiembre de 2016 y octubre de 2018.

**NOTA** Los datos han pasado por un proceso relativamente simple de curación, pero se han dejado algunos ruidos en la calidad de éstos con el fin de que los equipos también lleven a cabo un proceso de inspección y limpieza.


## NOTA IMPORTANTE

Recuerden que esta información aún contiene un elevado nivel de ruido. No solo no ha sido depurada de posibles datos atípicos (transacciones de valor muy elevado) fruto de errores o transacciones fallidas, sino que también cuenta con el ruido asociado al campo de referencia, donde se involucra el factor humano, ya que son campos de texto libre que pueden contener cualquier tipo de información.

Por seguridad, hemos eliminado cualquier número presente en dichos campos de referencia (cédulas, nits, montos, contratos, etc.).


In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import random
import time
%matplotlib inline
import matplotlib.animation as animation
import csv
matplotlib.style.use('seaborn')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn import manifold
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster import hierarchy
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import pairwise_distances
# from wordcloud import WordCloud

from mpl_toolkits.mplot3d import Axes3D

Cargar los datos proeprocesados de las bases de datos

In [11]:
pagadores=pd.read_csv("info_pagadores.csv",index_col=0)
pagadores.head()

,id_cliente,seg_str,ocupacion,tipo_vivienda,nivel_academico,estado_civil,genero,edad,ingreso_rango
0,18,PERSONAL PLUS,5,O,U,M,M,92.0,e. (4.4 5.5MM]
1,32,PERSONAL PLUS,E,F,T,M,M,80.0,i. (8.7 Inf)
2,41,EMPRENDEDOR,3,O,I,W,M,90.0,b. (1.1 2.2MM]
3,47,EMPRENDEDOR,7,NaN,I,I,M,86.0,c. (2.2 3.3MM]
4,71,PERSONAL,5,O,S,M,M,79.0,e. (4.4 5.5MM]


In [12]:
transac_pag=pd.read_csv("client_fr_val.csv",index_col=0)
transac_pag.head()

,id_cliente,count,valor_trx_suma
0,1,26,42769383.24
1,2,6,1365034.29
2,3,54,14180802.05
3,4,7,2534093.31
4,5,27,10990697.40


In [13]:
client_augm=pd.merge(pagadores, transac_pag, on='id_cliente')
client_augm.head(25)

,id_cliente,seg_str,ocupacion,tipo_vivienda,nivel_academico,estado_civil,genero,edad,ingreso_rango,count,valor_trx_suma
0,18,PERSONAL PLUS,5,O,U,M,M,92.0,e. (4.4 5.5MM],23,14235446.68
1,32,PERSONAL PLUS,E,F,T,M,M,80.0,i. (8.7 Inf),2,34069.44
2,41,EMPRENDEDOR,3,O,I,W,M,90.0,b. (1.1 2.2MM],21,5462187.17
3,47,EMPRENDEDOR,7,NaN,I,I,M,86.0,c. (2.2 3.3MM],42,11528966.59
4,71,PERSONAL,5,O,S,M,M,79.0,e. (4.4 5.5MM],50,14366272.24
5,72,PERSONAL,1,O,U,M,M,74.0,b. (1.1 2.2MM],22,945919.37
6,102,PREFERENCIAL,9,O,U,M,M,86.0,i. (8.7 Inf),18,16936316.99
7,108,PERSONAL PLUS,1,O,U,M,F,37.0,f. (5.5 6.6MM],12,2073818.63
8,210,EMPRENDEDOR,3,NaN,U,M,M,34.0,i. (8.7 Inf),53,57359637.28
9,226,EMPRENDEDOR,3,R,T,F,M,56.0,h. (7.6 8.7MM],4,7835798.78


In [9]:
client_augm.dtypes

id_cliente           int64
seg_str             object
ocupacion           object
tipo_vivienda       object
nivel_academico     object
estado_civil        object
genero              object
edad               float64
ingreso_rango       object
count                int64
valor_trx_suma     float64
dtype: object